In [1]:
!pip install evaluate sacrebleu

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/df/c0/ff53cb76c1b050ad25d056877ba6d3f6fa964134370c4ccf57ad933d6f72/sacrebleu-2.3.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 2.5 MB/s eta 0:00:00
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 11.5 MB/s eta 0:00:00


In [2]:
from transformers import (AutoTokenizer, DataCollatorForSeq2Seq,
AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer)
from datasets import load_dataset
from huggingface_hub import notebook_login

import numpy as np
import evaluate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
notebook_login()

In [34]:
def preprocess_function(examples):
    inputs = [PREFIX + example[SRC_LANG] for example in examples["translation"]]
    targets = [example[TGT_LANG] for example in examples["translation"]]
    model_inputs = tokenizer(inputs
                             , text_target = targets
                             , max_length = MAX_LENGTH
                             , padding = "max_length"
                             , truncation = True)
    return model_inputs


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    #print(preds)
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    #preds = [tid for tid in preds.flatten() if tid >= 0 and tid < tokenizer.vocab_size]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [5]:
MODEL_CHECKPOINT = "t5-base"
SRC_LANG = "en"
TGT_LANG = "de"
PREFIX = "translate English to German: "
MAX_LENGTH = 128

In [6]:
raw_data = load_dataset("opus_books", lang1=TGT_LANG, lang2=SRC_LANG)

Generating train split:   0%|          | 0/51467 [00:00<?, ? examples/s]

Dataset opus_books downloaded and prepared to /root/.cache/huggingface/datasets/opus_books/de-en-lang1=de,lang2=en/0.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
split_data = raw_data["train"].train_test_split(train_size=0.9, seed=0)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

In [35]:
token_data = split_data.map(preprocess_function, batched=True)

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer
                                       , model = MODEL_CHECKPOINT)

In [12]:
metric = evaluate.load("sacrebleu")

In [38]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "t5-base_fine-tuned_opus-books_en-de",
    evaluation_strategy = "epoch",
    learning_rate = 5e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 3,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = token_data["train"],
    eval_dataset = token_data["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

In [39]:
trainer.evaluate(max_length=MAX_LENGTH)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 13.984762191772461,
 'eval_bleu': 11.8599,
 'eval_gen_len': 36.3622,
 'eval_runtime': 400.1844,
 'eval_samples_per_second': 12.862,
 'eval_steps_per_second': 0.402}

In [40]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.646300,0.550666,5.306300,17.027000
2,0.598000,0.536998,5.470800,17.026400
3,0.581800,0.534076,5.559700,17.026200


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will i

TrainOutput(global_step=4344, training_loss=0.6497323297883485, metrics={'train_runtime': 5923.0305, 'train_samples_per_second': 23.461, 'train_steps_per_second': 0.733, 'total_flos': 2.11551971180544e+16, 'train_loss': 0.6497323297883485, 'epoch': 3.0})

In [41]:
trainer.evaluate(max_length=MAX_LENGTH)

{'eval_loss': 0.5340757369995117,
 'eval_bleu': 17.263,
 'eval_gen_len': 38.3748,
 'eval_runtime': 446.7223,
 'eval_samples_per_second': 11.522,
 'eval_steps_per_second': 0.36,
 'epoch': 3.0}

In [42]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

'https://huggingface.co/jaymanvirk/t5-base_fine-tuned_opus-books_en-de/tree/main/'